This will be a helper for analyzing grocery lists. This uses excel lists made in a specific format based off of GIANT, which may obtainable from scanning receipts.

First we set up the prerequisites and import the list file.

In [23]:
# Set up prerequisites
import pandas as pd
import math
# Import the file into pandas, works for current format.
f=pd.read_excel('list0.xlsx',header=3)

In [24]:
type(f.head())

pandas.core.frame.DataFrame

Just testing to see if it's working.

## Filtering

Now it's time for some cleanup. I'd like to first change the NaN values to blank values for readability.

In [25]:
for col in list(f):
    for ii in range(len(f[col])):
        f_ii=f.loc[ii,col]
        try:
            if math.isnan(f_ii):
                f.loc[ii,col]=''
        except:
            # Usually want to suppress this
            # print('Exception at ' + str(ii))
            continue

In [26]:
f.head()

,Name,S?,Pre-Sale Price,Unit Price,Qty.,Unit,Net Cost,Type,Brand
0,Impossible Burger Meat,S,,0.75,12,oz.,9,,Impossible Brand
1,Thick Cut Sliced Bacon,S,,3.99,1,lb.,3.99,,
2,Italian Herbs Stir-in Paste,S,,3.99,1,ct.,3.99,,
3,Good Butter,S,,0.49,8,oz.,3.92,,Vermont Creamery
4,Good Mozzarella,S,,0.62,8,oz.,4.96,,BelGioioso


Looking better. It's also important to have a 'special' category, for outlier purchases and for scalability sake. Those should be filtered out of the list. Does not to be run if no special category present, but can be without issue.

In [44]:
special=f[f['S?']!='']; special=special.drop('S?', axis=1);
f0=f[f['S?']=='']; f0=f0.drop('S?', axis=1);

In [45]:
f0.head()

,Name,Pre-Sale Price,Unit Price,Qty.,Unit,Net Cost,Type,Brand
5,Bake Shop,,,,,,,
6,Traditional Pizza Crust,4.99,3.5,1,ct.,3.5,,
7,Donut Crossiant,,0.7,2,ct.,1.4,,
8,Dairy,,,,,,,
9,Shredded Mozzarella,,0.25,16,oz.,4,,


GIANT stores have specific entry types depending on product, which are printed in bold above all items in that category. The following code is based on that type of entry.

There are bound to be inconsistencies between different stores, but larger pool of data provides allows for merging of store-specific types, or better definition along those types. It also allows for improved entry schema, ultimate goal of receipt scanning.